## Maestría en Analítica de datos
## Materia Big Data
## Clase 2

*   Nombre: José Luis Ariza
*   Código: 80125197


# 1.Cargar librerias

In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
url_forder_csv="/content/drive/MyDrive/Ucentral/2025S2/BigData/Clases/Tutelas"

# 2. carga/obtención de información

In [14]:
listado_archivos= [f for f in os.listdir(url_forder_csv) if f.endswith(".csv")]
print(listado_archivos)

['tutelas_con_providencias_2025.csv', 'tutelas_con_providencias_2024.csv']


In [15]:
columnas_nombre=[
    "clasificacion",
    "num_expediente",
    "fecha_Radica",
    "demandante",
    "demandado",
    "primera_instancia",
    "segunda_instancia",
    "num_23Digitos",
    "providencia",
    "fechaSentencia"
]
df_tutelas=pd.DataFrame(columns=columnas_nombre)
display (df_tutelas)

,clasificacion,num_expediente,fecha_Radica,demandante,demandado,primera_instancia,segunda_instancia,num_23Digitos,providencia,fechaSentencia


##2.a procesamiento secuencial (uno tras otro)

In [16]:
df_list=[]  #lista de df resultante de los datos de cada uno de los CSV
for archivo_nombre in listado_archivos:
  ruta_archivo= os.path.join(url_forder_csv,archivo_nombre) #ruta completa del archivo
  print(ruta_archivo)

  df_temporal = pd.read_csv(ruta_archivo,sep=';',header=None,encoding='latin-1',on_bad_lines='skip')
  #agregarle nombre de columnas al df creado
  df_temporal.columns=columnas_nombre

  if (len(df_temporal.columns)==len(columnas_nombre)):
    df_list.append(df_temporal)
    print(f" archivo {archivo_nombre} cargada exitosamente")
  else:
    print(f"archivo no trabajado {archivo_nombre} no tiene 10 columnas")
#--------una vez termine de cargar todos los CSV combinelos en el dataframa df_tutelas
if df_list:
  df_tutelas= pd.concat(df_list,ignore_index=True)
  # reemplazar las palabras "NULL" por none
  df_tutelas.replace("NULL",None,inplace=True)
  display(df_tutelas.head())


/content/drive/MyDrive/Ucentral/2025S2/BigData/Clases/Tutelas/tutelas_con_providencias_2025.csv


/tmp/ipython-input-3669624718.py:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temporal = pd.read_csv(ruta_archivo,sep=';',header=None,encoding='latin-1',on_bad_lines='skip')


 archivo tutelas_con_providencias_2025.csv cargada exitosamente
/content/drive/MyDrive/Ucentral/2025S2/BigData/Clases/Tutelas/tutelas_con_providencias_2024.csv
 archivo tutelas_con_providencias_2024.csv cargada exitosamente


,clasificacion,num_expediente,fecha_Radica,demandante,demandado,primera_instancia,segunda_instancia,num_23Digitos,providencia,fechaSentencia
0,ï»¿Acciones de Tutela,T0793175,2033-09-10 00:00:00,AGUILAR MURILLO WILSON EMILIO,SECRETARIA DE EDUCACION DEPARTAMENTAL,BUENAVENTURA JUZGADO 1 CIVIL DEL CTO,--,--,--,NaN
1,Acciones de Tutela,T11446144,2025-09-01 00:00:00,CHAVES PORRAS ANA SOFIA,EPS FAMISANAR,"BOGOTA DC, BOGOTA DC, JUZGADO 116 PENAL MUNICI...",NaN,11001400911620250016100,--,NaN
2,Acciones de Tutela,T11446145,2025-09-01 00:00:00,KEEP MORALES HENRY,INSTITUTO COLOMBIANO DE BIENESTAR FAMILIAR,"PALMIRA, VALLE DEL CAUCA, JUZGADO 003 DE EJECU...","GUADALAJARA DE BUGA, VALLE DEL CAUCA, TRIBUNAL...",76520318700320250006500,--,NaN
3,Acciones de Tutela,T11446142,2025-09-01 00:00:00,AVILA GONZALEZ ELVIN,CONSORCIO PAVEX,"MEDELLIN, ANTIOQUIA, JUZGADO 029 PENAL MUNICIP...","MEDELLIN, ANTIOQUIA, JUZGADO 012 PENAL DEL CIR...",05001408802920250022200,--,NaN
4,Acciones de Tutela,T11446143,2025-09-01 00:00:00,BANCO SANTANDER DE NEGOCIOS COLOMBIA SA,FISCALIA GENERAL DE LA NACION,"ITAGUI, ANTIOQUIA, JUZGADO 001 CIVIL DEL CIRCU...",NaN,05360310300120250033400,--,NaN


##2.b procesamiento en paralelo (todos al tiempo)

In [17]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [18]:
def cargar_data_desde_archivo_csv(ruta_archivo):
  try:
      df_temporal = pd.read_csv(ruta_archivo,sep=';',header=None,encoding='latin-1',on_bad_lines='skip')
      #agregarle nombre de columnas al df creado
      df_temporal.columns=columnas_nombre
      if (len(df_temporal.columns)==len(columnas_nombre)):
        print(f" archivo {os.path.basename(ruta_archivo)} cargada exitosamente")
        return df_temporal
      else:
        print(f"archivo no trabajado {os.path.basename(ruta_archivo)} no tiene 10 columnas")
        return None

  except Exception as e:
      print(f"Error al procesar el archivo {os.path.basename(ruta_archivo)}: {str(e)}")
      return None

In [19]:
df_list=[]  #lista de df resultante de los datos de cada uno de los CSV
with ThreadPoolExecutor() as executor:
  future_to_file= {executor.submit(cargar_data_desde_archivo_csv,os.path.join(url_forder_csv,archivo_nombre)):archivo_nombre for archivo_nombre in listado_archivos}
  for future in as_completed(future_to_file):
    df=future.result()
    if df is not None:
      df_list.append(df)
#--------una vez termine de cargar todos los CSV combinelos en el dataframa df_tutelas
if df_list:
  df_tutelas= pd.concat(df_list,ignore_index=True)
  # reemplazar las palabras "NULL" por none
  df_tutelas.replace("NULL",None,inplace=True)
  display(df_tutelas.head())


/tmp/ipython-input-3430010226.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temporal = pd.read_csv(ruta_archivo,sep=';',header=None,encoding='latin-1',on_bad_lines='skip')


 archivo tutelas_con_providencias_2025.csv cargada exitosamente
 archivo tutelas_con_providencias_2024.csv cargada exitosamente


,clasificacion,num_expediente,fecha_Radica,demandante,demandado,primera_instancia,segunda_instancia,num_23Digitos,providencia,fechaSentencia
0,ï»¿Acciones de Tutela,T0793175,2033-09-10 00:00:00,AGUILAR MURILLO WILSON EMILIO,SECRETARIA DE EDUCACION DEPARTAMENTAL,BUENAVENTURA JUZGADO 1 CIVIL DEL CTO,--,--,--,NaN
1,Acciones de Tutela,T11446144,2025-09-01 00:00:00,CHAVES PORRAS ANA SOFIA,EPS FAMISANAR,"BOGOTA DC, BOGOTA DC, JUZGADO 116 PENAL MUNICI...",NaN,11001400911620250016100,--,NaN
2,Acciones de Tutela,T11446145,2025-09-01 00:00:00,KEEP MORALES HENRY,INSTITUTO COLOMBIANO DE BIENESTAR FAMILIAR,"PALMIRA, VALLE DEL CAUCA, JUZGADO 003 DE EJECU...","GUADALAJARA DE BUGA, VALLE DEL CAUCA, TRIBUNAL...",76520318700320250006500,--,NaN
3,Acciones de Tutela,T11446142,2025-09-01 00:00:00,AVILA GONZALEZ ELVIN,CONSORCIO PAVEX,"MEDELLIN, ANTIOQUIA, JUZGADO 029 PENAL MUNICIP...","MEDELLIN, ANTIOQUIA, JUZGADO 012 PENAL DEL CIR...",05001408802920250022200,--,NaN
4,Acciones de Tutela,T11446143,2025-09-01 00:00:00,BANCO SANTANDER DE NEGOCIOS COLOMBIA SA,FISCALIA GENERAL DE LA NACION,"ITAGUI, ANTIOQUIA, JUZGADO 001 CIVIL DEL CIRCU...",NaN,05360310300120250033400,--,NaN
